In [2]:
library(data.table)
library(dplyr)
library(ggplot2)

In [3]:
refposs = fread(snakemake@input$ref_kept,header = F, col.names = c("Position"))
refposs$Lineage = snakemake@params$ref
idposs = fread(snakemake@input$id_kept,header = F,col.names = c("Position"))
idposs$Lineage = snakemake@params$id


In [4]:
bothdf <- bind_rows(refposs,idposs)
bothdf$KEPT <- 1
head(bothdf)

In [5]:
options(repr.plot.width=10, repr.plot.height=2, jupyter.plot_mimetypes = "image/svg+xml")

ggplot(data = bothdf) +
geom_point(aes(x=Position,y=KEPT, color =Lineage),size=0.1) +
ylim(0.5,1.5) +
labs(y="")+
guides(color = "none") +
facet_grid(rows = vars(Lineage)) +
 theme(axis.ticks.y = element_blank(),
        axis.text.y = element_blank())

In [6]:
intersects = intersect(refposs$Position,idposs$Position)
intersectposs = data.table(Position = intersects)
intersectposs$Lineage <- "Both"
intersectposs$KEPT <- 1

In [7]:
bothdf <- bind_rows(bothdf,intersectposs)
summarydf <- bothdf %>%
    group_by(Lineage) %>%
    summarise(Positions=n())
print(summarydf)

In [8]:
intersectposs$KEPT <- NULL
intersectposs$Lineage <- NULL
fwrite(file = snakemake@output$id_kept,x = intersectposs, col.names = F)